In [56]:
import asyncio
from dotenv import load_dotenv
load_dotenv()
import nest_asyncio
nest_asyncio.apply()
import os
from anthropic import AsyncAnthropic
from xml.sax.saxutils import escape
import re
import json

In [65]:
prompt = """\n\n
The above is a transcript of a lecture video with timestamps for each sentence in seconds.

Your task is to create a 30 second engaging and educational tiktok script for one topic in the video. 
Choose one of the more obscure and interesting topics from the transcript that most people dont know about.
The tiktok should incorporate an engaging story or example.
The script should be in ssml format. But do not change voices, only add pauses and emphasis.
"""

music_prompt = """
The above is a script for a tiktok video.
Please generate a short one sentence description of the music that should be playing in the background of the video.
Include genre and mood
Example:
"A short upbeat EDM tune with a catchy melody"
"""

In [58]:
# Load transcript from data/transcript.txt
with open("data/transcript.txt", "r") as f:
    transcript = f.read()

In [59]:
async def generate_text(text, script):
    client = AsyncAnthropic(
        # This is the default and can be omitted
        api_key=os.environ.get("ANTHROPIC_API_KEY"),
    )

    message = await client.messages.create(
        temperature=0,
        max_tokens=512,
        messages=[
            {
                "role": "user",
                "content": text + script,
            },
            {"role": "assistant", "content": "<"},
        ],
        model="claude-3-opus-20240229",
    )

    return message.content

response = asyncio.run(generate_text(transcript, prompt))

In [60]:
script = f"<{response[0].text}"

In [61]:
print(script)

<speak>
Have you ever wondered how your contacts app finds your friends' numbers so quickly? 🤔 It's thanks to a clever data structure called a <emphasis>hash table</emphasis>.

Imagine you're at a library, and you need to find a specific book. 📚 You could search through every single shelf, but that would take forever! Instead, libraries use the <emphasis>Dewey Decimal System</emphasis>, which is like a hash table for books.

In a hash table, your friends' names are organized into different "buckets" based on the first letter of their name. 🍯 So when you search for your friend "Emma", the app only needs to look in the "E" bucket, making the search <emphasis>lightning-fast</emphasis>! ⚡

The catch? If you have too many friends with the same first letter, the buckets can overflow, slowing things down. 😅 But most of the time, hash tables are a <emphasis>game-changer</emphasis> for speedy searches! 🎉

So the next time you find a contact in a flash, you can thank the power of hash tables! 😄


In [66]:
async def generate_text(text, script):
    client = AsyncAnthropic(
        # This is the default and can be omitted
        api_key=os.environ.get("ANTHROPIC_API_KEY"),
    )

    message = await client.messages.create(
        temperature=0,
        max_tokens=64,
        messages=[
            {
                "role": "user",
                "content": text + script,
            },
        ],
        model="claude-3-opus-20240229",
    )

    return message.content


response = asyncio.run(generate_text(script, music_prompt))
music = response[0].text

In [67]:
music

'Playful and quirky instrumental hip-hop track with a bouncy rhythm.'

In [63]:
with open("data/data.json", "w") as f:
    json.dump({"script": script, "music_description": music}, f, indent=4)